In [37]:
from dotenv import load_dotenv

load_dotenv()

True

In [38]:
from langchain.schema import Document
import json

# 파일 경로 
file_paths = {
    "term": "./metadata/term.json",
    "load_traffic_law": "./metadata/load_traffic_law.json",
    "modifier": "./metadata/modiflier.json",
    "car_case": "./metadata/car_to_car.json",
    "precedent": "./metadata/law.json"
}

# 교통사고 케이스용 필드 상수
CASE_ID = "사건 ID"
CASE_TITLE = "사건 제목"
CASE_SITUATION = "사고상황"
BASE_RATIO = "기본 과실비율"
MODIFIERS = "케이스별 과실비율 조정예시"
LAW_REFERENCES = "관련 법규"
PRECEDENT = "참고 판례"
REASON = "기본 과실비율 해설"

# JSON 로드 함수
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 리스트형 JSON 변환 (term, modifier, law_meta)
def convert_list_to_documents(data_list, doc_type):
    return [
        Document(page_content=json.dumps(item, ensure_ascii=False), metadata={"type": doc_type})
        for item in data_list
    ]

def convert_precedent_documents(data_list):
    return [
        Document(
            page_content=f"{item['court']} {item['case_id']} : {item['content']}",
            metadata={
                "court": item["court"],
                "case_id": item["case_id"],
            }
        ) for item in data_list
    ]

def convert_car_case_documents(data_list):
    documents = []

    def safe_value(value):
        if isinstance(value, list):
            return ", ".join(map(str, value))
        elif isinstance(value, dict):
            return json.dumps(value, ensure_ascii=False)
        elif value is None:
            return ""  # null도 허용 안 되므로 빈 문자열로 처리
        else:
            return str(value)

    for item in data_list:
        if not isinstance(item, dict):
            continue

        # page_content는 원본 전체 JSON 문자열
        content = json.dumps(item, ensure_ascii=False)

        # 기본 과실비율 해설이 리스트일 수 있음 → 문자열로 병합
        reason = item.get(REASON)
        if isinstance(reason, list):
            reason = "\n".join(map(str, reason))

        metadata = {
            "type": "car_case",
            "id": safe_value(item.get(CASE_ID)),
            "title": safe_value(item.get(CASE_TITLE)),
            "situation": safe_value(item.get(CASE_SITUATION)),
            "base_ratio": safe_value(item.get(BASE_RATIO)),
            "modifiers": safe_value(item.get(MODIFIERS)),
            "load_traffic_law": safe_value(item.get(LAW_REFERENCES)),
            "precedent": safe_value(item.get(PRECEDENT)),
            "reason": safe_value(reason)
        }

        documents.append(Document(page_content=content, metadata=metadata))
    return documents

# 도로교통법 law JSON → 문서화
def convert_law_json_to_documents(data_dict):
    documents = []

    def normalize(item):
        return json.dumps(item, ensure_ascii=False) if isinstance(item, dict) else str(item)

    for law_name, content in data_dict.items():
        if isinstance(content, dict):
            for clause, text in content.items():
                lines = [normalize(x) for x in (text if isinstance(text, list) else [text])]
                full_text = f"{law_name} {clause}\n" + "\n".join(lines)
                documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
        else:
            lines = [normalize(x) for x in (content if isinstance(content, list) else [content])]
            full_text = f"{law_name}\n" + "\n".join(lines)
            documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
    
    return documents

def convert_judgment_documents(data_list):
    documents = []

    for item in data_list:
        if not isinstance(item, dict):
            continue

        court = item.get("court", "").strip()
        case_id = item.get("case_id", "").strip()
        content = item.get("content", "").strip()

        if not court or not case_id or not content:
            continue

        # 💡 판례명 + 콜론 + 내용 구조
        page_content = f"{court} {case_id} : {content}"

        metadata = {
            "type": "judgment",
            "court": court,
            "case_id": case_id
        }

        documents.append(Document(page_content=page_content, metadata=metadata))

    return documents


# --- 각 문서 변환 실행 ---
term_docs = convert_list_to_documents(load_json(file_paths["term"]), "term")
modifier_docs = convert_list_to_documents(load_json(file_paths["modifier"]), "modifier")
car_case_docs = convert_car_case_documents(load_json(file_paths["car_case"]))
load_traffic_law_docs = convert_law_json_to_documents(load_json(file_paths["load_traffic_law"]))
judgment_docs = convert_judgment_documents(load_json(file_paths["precedent"]))  # ✅ 핵심

# --- 전체 문서 조합 ---
USE_ALL = False  # True로 바꾸면 전체 문서를 포함하여 사용 가능

all_docs = (
    term_docs + modifier_docs + car_case_docs + load_traffic_law_docs + judgment_docs
    if USE_ALL else judgment_docs
)


# # 문서화 실행
# term_docs = convert_list_to_documents(load_json(file_paths["term"]), "term")
# modifier_docs = convert_list_to_documents(load_json(file_paths["modifier"]), "modifier")
# precedent_docs = convert_precedent_documents(load_json(file_paths["precedent"]))
# car_case_docs = convert_car_case_documents(load_json(file_paths["car_case"]))
# load_traffic_law_docs = convert_law_json_to_documents(load_json(file_paths["load_traffic_law"]))


# # 전체 문서 리스트
# #all_docs = term_docs + modifier_docs + car_case_docs + precedent_docs + load_traffic_law_docs
# all_docs = precedent_docs

In [39]:
from langchain.vectorstores import Chroma  # persist 지원
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 청크 크기 조정 (500~1000 권장)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# 2. 문서 분할
all_splits = text_splitter.split_documents(all_docs)

embedding_model = OpenAIEmbeddings()

# 4. Chroma DB에 배치 처리로 저장
batch_size = 100  # 한 번에 처리할 청크 수
vectorstore = Chroma.from_documents(
    documents=all_splits[:batch_size],  # 첫 배치
    embedding=embedding_model,
    persist_directory="./vectordb"
)

# 남은 청크를 순차적으로 추가
for i in range(batch_size, len(all_splits), batch_size):
    batch = all_splits[i:i+batch_size]
    vectorstore.add_documents(
        documents=batch,
        embedding=embedding_model
    )

vectorstore.persist()  


In [40]:
vectorstore.get()

{'ids': ['7e84b59c-7140-47dc-9b7c-f3a4608f5fa0',
  '8d7d7350-5942-46ef-9fad-19f01dc59f77',
  'ffa95a58-4ad7-4107-ae13-c622e5e45a9b',
  '90f413c4-5578-42ed-877b-f996e82f0d6d',
  '2412fde0-8fce-43e4-9b9c-235ecb21f90c',
  '5175ad97-abad-4c02-9c26-907e06f181fe',
  '8e190bb3-2d3c-4cc1-a26d-54b7380e607d',
  '608298ea-7875-4342-a4f7-aa97310c5b4c',
  '7fb8f92d-e4c2-4173-a910-ffc55b4c1b27',
  'd41597a2-a2e9-46e5-b340-1a4fdc71c03c',
  'da83cfe7-a92b-4ac4-8f83-5263dfac3ef2',
  '0e7bc5ac-d0a0-4f49-aab8-6dce98a9bebb',
  'ae989ea3-fab9-4edb-a8f5-1a7591abb02f',
  'dad2a8f7-2ef3-41b5-b000-3af45f00f172',
  'd6166fe2-b795-481a-a9c6-24ca655dbf7e',
  'e4585a21-92f5-46d4-9c0b-515ec231d5e3',
  '3af5c6e7-7033-463b-ba82-85090c107979',
  '430a2912-d577-4f20-a0d0-a2cc58bd53b3',
  'a28b1276-431b-47a6-b2cf-7b143717fbb5',
  '8a8e2071-a76f-45fb-a0d0-431a6783769c',
  '5e4265f0-6317-44e6-ae29-08d5f80ba309',
  '66d740b9-0e72-4560-b69c-45103ac3e6cd',
  '9a788ede-0818-4f1d-8778-0cd4c931f66c',
  'c853bf49-cd61-433d-b13b-

In [41]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma

# 1. LLM 설정
model = ChatOpenAI(model='gpt-4o', temperature=0.5)

# 2. 프롬프트 템플릿
prompt = PromptTemplate(
    template="""
다음 '사고 상황 설명'에 대해 '문서'의 내용만 참고하여 과실 비율 및 법적 판단을 생성해 주세요.

사고 상황: {question}

문서 내용: {context}

답변 형식:
1. 과실비율: A차량 xx% vs B차량 xx%
2. 사고 원인 및 판단 근거:
   - [사고의 전개, 각 차량의 행위, 과실 요소 등을 구체적으로 설명]
3. 관련 법률 조항:
   - [예: 도로교통법 제10조 제2항, 제27조 제1항 등]
4. 참고 판례:
   - [예: 대법원 2023다12345, 서울중앙지법 2022가단123456 등]

조건:
- 반드시 문서 내용만 참고해 판단하세요.
- 법률 조항과 판례가 명시되어 있지 않으면 유사하거나 추정 가능한 근거를 제시해도 됩니다.
- 전체 답변은 포맷에 맞게 간결하고 전문적으로 작성하세요.
""",
    input_variables=["question", "context"]
)

# 3. 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# 4. QA 체인 재구성 (수정 부분)
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# 5. 실제 질의 실행
query = "B차량이 비보호좌회전 표지가 없는 교차로에서 녹색등(좌회전 화살표 없음)에 좌회전 시도, 직진하는 A차량과 충돌한 사고가 발생했어. 과실비율이 어떻게 돼??"

res = qa_chain.invoke({"query": query})

# 6. 출력
print("✅ 답변:\n", res["result"])


✅ 답변:
 1. 과실비율: A차량 20% vs B차량 80%

2. 사고 원인 및 판단 근거:
   - 사고는 비보호좌회전 표지가 없는 교차로에서 발생했으며, B차량이 녹색등에 좌회전을 시도하던 중 직진하는 A차량과 충돌했습니다. 
   - 서울중앙지방법원 2017나65373 판례에 따르면, 비보호좌회전 시 좌회전 차량은 직진 차량의 진행을 방해하지 않도록 주의할 의무가 있습니다. 
   - 이 판례에서는 좌회전 차량의 과실을 80%로 판단했습니다. 따라서 B차량이 비보호좌회전 시 직진 차량인 A차량의 진행을 방해한 점을 고려하여 B차량의 과실을 80%로 판단합니다.

3. 관련 법률 조항:
   - 도로교통법 제25조 제2항: 좌회전 시 반대 방향에서 오는 차량의 진로를 방해하지 않을 의무
   - 도로교통법 제27조 제1항: 교차로에서의 통행 우선권

4. 참고 판례:
   - 서울중앙지방법원 2017나65373: 비보호좌회전 차량의 과실을 80%로 인정한 사례.


In [42]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

detail_prompt = PromptTemplate(
    template="""
아래 문서 내용을 바탕으로 사용자가 물어본 용어나 법률 조항, 판례에 대해 정확하고 간결하게 설명해 주세요.

질문: {question}

문서: {context}

답변 형식:
- 용어/조항 정의: [정확한 설명]
- 출처가 명시된 경우: 관련 법률/조문 번호/판례명을 반드시 포함

답변:
""",
    input_variables=["question", "context"]
)

detail_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": detail_prompt}
)

# run은 question 키를 기준으로 실행됨!
res2 = detail_chain.run("보행자우선도로가 뭔지 설명해줘")

print("✅ 기능② 답변:\n", res2)


✅ 기능② 답변:
 - 용어/조항 정의: 보행자우선도로는 보행자의 안전과 편의를 우선적으로 고려하여 설계된 도로를 의미합니다. 이 도로에서는 보행자가 우선적으로 통행할 수 있으며, 차량은 보행자의 안전을 최우선으로 하여 서행하거나 정지해야 할 의무가 있습니다. 보행자우선도로는 보행자와 차량 간의 사고를 예방하고 보행 환경을 개선하기 위해 도입된 제도입니다.


In [43]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import re

# LLM 설정
model = ChatOpenAI(model='gpt-4o', temperature=0.5)

# 기능 1번. 사고 상황 판단 프롬프트
prompt = PromptTemplate(
    template="""
너는 교통사고 과실 판단 전문가야.
아래 '사고 상황'을 분석하여 핵심 요소를 구조화하고, 반드시 문서 내에서 가장 유사한 사례(case)를 찾아 과실비율을 판단해줘.

---
사고 상황 원문:
{question}

➤ 사고 상황 요약 (다음 항목 기준):
- A차량 신호 및 진행 방식:
- B차량 신호 및 진행 방식:
- 충돌 방식 및 위치:
- 교차로/신호기 유무 등 도로 환경:

문서:
{context}

출력 형식:
1. 과실비율: A차량 xx% vs B차량 xx%
2. 판단 근거 요약
3. 적용 법률 (다음 형식으로 기재)
   - [법률명] 제[조]조 [항] (예: 도로교통법 제5조 제1항)
   - 여러 개면 각각 별도 줄로 나열
4. 참고 판례 (다음 형식으로 기재)
   - [법원명] [사건번호] (예: 대법원 2002다38767)
   - 여러 개면 각각 별도 줄로 나열

조건:
- 반드시 문서 내 유사 사례를 기반으로 판단해야 해.
- 사고 상황과 가장 유사한 사례를 먼저 정확히 찾고, 그 사례를 중심으로 판단 근거를 제시해.
- 직접적인 사례가 없으면, 가장 유사한 사례를 근거로 판단하고 그 차이를 설명해.
- 과실비율은 명확한 수치로 제시해.
- 법령, 판례는 반드시 문서에서 인용된 것을 정확한 형식으로 명시해.
- 무조건적인 추론이나 일반적인 상식은 사용하지 말고, 문서에 있는 정보로만 판단해.
- 판례명을 포함한 질문인 경우, 문서에서 사건번호와 법원명이 일치하는 문장을 찾아 정확히 인용해줘.
""",
    input_variables=["question", "context"]
)


# 기능 2번. 용어/법률 설명 프롬프트
detail_prompt = PromptTemplate(
    template="""
아래 문서 내용을 바탕으로 사용자가 물어본 용어나 법률 조항, 판례에 대해 정확하고 간결하게 설명해 주세요.

질문: {question}

문서: {context}

답변 형식:
- 용어/조항 정의: [정확한 설명]
- 출처가 명시된 경우: 관련 법률/조문 번호/판례명을 반드시 포함

조건:
- 질문에 판례명(법원명 + 사건번호)이 포함된 경우, 정확히 해당 판례가 포함된 문서를 찾아 내용을 인용하여 설명해줘.
- 문서에 없다면 판례가 존재하지 않는다고 판단해.

""",
    input_variables=["question", "context"]
)

# 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 8})

# 사고 상황 판단 체인
accident_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

# 설명용 체인
detail_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": detail_prompt}
)

def classify_query(user_input: str) -> str:
    accident_keywords = ["충돌", "사고", "과실", "A차량", "B차량", "신호위반", "부딪힘", "진입", "교차로"]
    explanation_keywords = ["무엇", "뭐야", "정의", "설명", "판례", "조항", "법원", "고등", "법", "도로교통법" ,"무슨 뜻", "의미", "용어", "이란"]

    # 기능② 설명 우선 처리
    if any(word in user_input for word in explanation_keywords):
        return "detail"
    if any(word in user_input for word in accident_keywords):
        return "accident"
    return "detail"


def extract_case_id(text: str):
    match = re.search(r"(대법원|서울고등법원|서울중앙지방법원|[\w]+지방법원)\s?\d{4}[가-힣]?\d+", text)
    return match.group() if match else None



def run_query(user_input: str):
    case_id = extract_case_id(user_input)

    if case_id:
        matched_docs = [doc for doc in all_docs if case_id in doc.page_content]

        if matched_docs:
            result = qa_chain.run({"query": user_input, "context": matched_docs})
        else:
            result = f"❌ 해당 판례 '{case_id}'는 문서에 포함되어 있지 않습니다."
    else:
        result = qa_chain.run(user_input)

    print("✅ 답변:\n", result)


In [44]:
print("총 저장된 문서 수:", len(vectorstore.similarity_search("", k=1000)))

총 저장된 문서 수: 1000


In [45]:
run_query("A차량이 적색신호에 직진 중이고, B차량은 좌회전 신호 없이 녹색등에서 좌회전 중 충돌한 사고야. 과실비율은?")

ValueError: `run` not supported when there is not exactly one output key. Got ['result', 'source_documents'].

In [ ]:
run_query("A차량과 A차량의 진행방향 왼쪽 도로에서 진입하여 좌회전 하는 B차량이 충돌한 사고야. 과실비율이 어떻게 돼?")

✅ 답변:
 1. 과실비율: A차량 20% vs B차량 80%

2. 사고 원인 및 판단 근거:
   - 사고는 A차량이 사거리 교차로에서 좌회전하던 중 A차량의 진행방향 왼쪽 도로에서 진입하여 좌회전하던 B차량과 충돌하면서 발생했습니다.
   - 문서에 따르면, 유사한 사고에서 A차량이 좌회전 중 B차량과 충돌한 경우 A차량의 과실비율이 20%로 책정되었습니다. 따라서 A차량의 과실은 20%로 추정할 수 있습니다.
   - B차량은 A차량의 진행방향 왼쪽 도로에서 진입하여 좌회전하던 중 충돌이 발생했으며, 이는 교차로에서의 주의 의무를 다하지 못한 것으로 볼 수 있습니다.

3. 관련 법률 조항:
   - 도로교통법 제27조 제1항: 모든 운전자는 교차로에서 회전할 때 주의 의무를 다해야 합니다.

4. 참고 판례:
   - 서울중앙지방법원 2017나63155: A차량 20% 과실 판례를 참고하여 유사한 사고 상황에서의 과실비율을 적용하였습니다.


In [ ]:
run_query("비보호 좌회전이 뭔지 설명해줘")

✅ 답변:
 답변:

1. 과실비율: A차량 80% vs B차량 0%

2. 사고 원인 및 판단 근거:
   - 비보호 좌회전 구역에서 좌회전하는 A차량은 반대방향에서 진행신호에 따라 직진하는 B차량의 통행을 방해하지 않도록 조심스럽게 좌회전을 해야 할 의무가 있습니다. 그러나 A차량은 교차로 맞은편에서 이미 차로에 진입하고 있던 B차량을 충격하였습니다. 이러한 상황에서 A차량의 과실이 80%로 판단됩니다. B차량은 정상 신호에 따라 직진 중이었으며, 특별한 주의의무를 위반한 사정이 없었습니다.

3. 관련 법률 조항:
   - 도로교통법 제27조 제1항: 모든 차량은 교차로에서 좌회전 시 반대 방향에서 오는 차량의 주행을 방해하지 않도록 해야 한다.

4. 참고 판례:
   - 서울중앙지방법원 2017나65373
   - 대구지방법원 2016나309440

위의 내용은 문서에 명시된 내용에 따라 판단된 것이며, 실제 법적 판단은 구체적인 사고 정황 및 추가 증거에 따라 달라질 수 있습니다.


In [ ]:
run_query("도로교통법 제5조(신호에 따른 통행의무) 1항이 뭐야?")

✅ 답변:
 1. 과실비율: A차량 0% vs B차량 100%

2. 사고 원인 및 판단 근거:
   - 사고는 신호등에 의하여 교통정리가 행하여지는 교차로에서 발생했습니다. A차량은 진행신호에 따라 정상적으로 교차로를 통과하고 있었으며, 이러한 상황에서는 다른 차량들도 교통법규를 준수할 것이라 믿고 운전하는 것이 일반적입니다. 반면, B차량은 신호를 위반하여 A차량의 진로를 가로질러 진행하였고, 이로 인해 충돌 사고가 발생했습니다. 문서에 따르면, A차량 운전자는 B차량의 신호 위반과 같은 상황을 미리 방지할 특별한 주의의무가 없으므로, A차량의 과실은 없다고 판단됩니다.

3. 관련 법률 조항:
   - 도로교통법 제5조(신호에 따른 통행의무) 1항

4. 참고 판례:
   - 대법원 99다30428
   - 대법원 2005다7177


In [ ]:
run_query("비보호 좌회전과 관련된 판례 알려줘")

✅ 답변:
 1. 과실비율: A차량 80% vs B차량 0%

2. 사고 원인 및 판단 근거:
   - 사고는 비보호 좌회전 구역에서 발생하였으며, A차량은 비보호 좌회전을 시도하던 중이었다. A차량 운전자는 반대 방향에서 정상 신호에 따라 직진하는 B차량의 진로를 방해하지 않도록 주의할 의무가 있었다.
   - A차량은 이러한 주의 의무를 다하지 못하고 교차로에 이미 진입한 B차량과 충돌하였다. 따라서 A차량의 과실이 80%로 판단된다.
   - B차량은 정상 신호에 따라 직진하고 있었으며, 특별히 주의 의무를 위반한 사정이 없으므로 과실이 없다.

3. 관련 법률 조항:
   - 도로교통법 제25조 제1항 (좌회전 시 반대 방향 직진 차량의 통행 방해 금지)
   - 도로교통법 제27조 제1항 (교차로 통행 방법)

4. 참고 판례:
   - 서울중앙지방법원 2017나65373: 비보호 좌회전 차량의 주의 의무와 과실 비율
   - 대구지방법원 2016나309440: 정상 신호에 따른 직진 차량의 무과실 인정


In [ ]:
run_query("서울중앙지방법원 2019나50924 알려줘")

✅ 답변:
 죄송하지만, 제공된 문서 내용에는 '서울중앙지방법원 2019나50924'에 대한 정보가 포함되어 있지 않습니다. 따라서 해당 사건에 대한 정확한 과실 비율 및 법적 판단을 생성할 수 없습니다. 다만, 문서에 포함된 사례를 참고하여 유사한 사고 상황에 대한 판단을 제시할 수 있습니다.

1. 과실비율: A자전거 40% vs B차량 60%

2. 사고 원인 및 판단 근거:
   - A자전거는 우측에서 중앙선을 침범하여 차로를 횡단한 후 역방향으로 진행하였고, 이는 도로교통법상 중앙선 침범 및 역주행에 해당하는 과실입니다.
   - B차량은 일방통행 도로를 역주행하여 교차로에 진입하였고, 이는 일방통행 위반에 해당하는 과실입니다.
   - B차량의 역주행이 사고의 주요 원인으로 판단되어 B차량에 더 높은 과실 비율이 부과되었습니다.

3. 관련 법률 조항:
   - 도로교통법 제13조 제1항: 중앙선 침범 금지
   - 도로교통법 제12조: 일방통행 위반 금지

4. 참고 판례:
   - 서울중앙지방법원 2015가단5001249: 유사한 사고 상황에서 B차량의 과실이 더 높게 인정된 사례

위 판단은 문서에 제공된 사례를 바탕으로 유사한 사고에 대한 일반적인 과실 비율 및 법적 판단을 제시한 것입니다. 정확한 사건에 대한 정보가 필요할 경우, 해당 사건의 판결문을 직접 확인하는 것이 필요합니다.


In [ ]:
run_query("서울중앙지방법원 2018나70853은 무슨 판례야?")

✅ 답변:
 죄송합니다. 주어진 문서 내용에는 서울중앙지방법원 2018나70853 판례에 대한 정보가 포함되어 있지 않습니다. 따라서 해당 사건에 대한 구체적인 과실 비율이나 법적 판단을 제시할 수 없습니다.

그러나, 문서에 명시된 사례를 바탕으로 일반적인 사고 판단 형식을 제공할 수 있습니다.

1. 과실비율: A차량 xx% vs B차량 xx% (2018나70853 사건에 대한 정보가 없어 비율을 제시할 수 없습니다.)
2. 사고 원인 및 판단 근거:
   - [사고의 전개, 각 차량의 행위, 과실 요소 등을 구체적으로 설명할 수 없습니다. 문서에 제공된 정보가 없습니다.]
3. 관련 법률 조항:
   - [관련 법률 조항을 제시할 수 없습니다. 문서에 제공된 정보가 없습니다.]
4. 참고 판례:
   - [관련 판례를 제시할 수 없습니다. 문서에 제공된 정보가 없습니다.]

이와 같은 경우, 사건에 대한 구체적인 정보를 확인하기 위해서는 해당 사건의 판결문을 직접 검토하거나 법률 전문가의 자문을 구하는 것이 필요합니다.
